In [1]:
from data_class import geo_Omsk
import os
import os.path as osp
from graphormer.data.wrapper import preprocess_item
from graphormer.data.collator import collator_graphormer
import torch
from graphormer.pretrain import load_pretrained_model
from pathlib import Path
from fairseq import checkpoint_utils, utils, options, tasks

Using backend: pytorch


In [10]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
split = 'train'
raw_data = geo_Omsk(raw_dir, split = split)

In [11]:
raw_data[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [12]:
preprocess_item(raw_data[0], 'dijkstra')

start find path dijkstra
dijkstra end with time 0.016786813735961914
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.011266708374023438


Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [13]:
preprocess_item(raw_data[0], 'floyd')

start find path floyd
floyd end with time 0.0018687248229980469
start gen_edge_input floyd
gen_edge_input floyd end with time 0.11255431175231934


Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [17]:
preprocess_item(raw_data[5])

start find path dijkstra
dijkstra end with time 0.16567063331604004
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.0701606273651123


Data(attn_bias=[122, 122], attn_edge_type=[121, 121, 1], edge_attr=[121, 1], edge_index=[2, 121], edge_input=[121, 121, 120, 1], in_degree=[121], out_degree=[121], spatial_pos=[121, 121], x=[121, 13], y=[1])

In [ ]:
for checkpoint_fname in os.listdir(args.save_dir):
            checkpoint_path = Path(args.save_dir) / checkpoint_fname
            model_state = torch.load(checkpoint_path)["model"]
            model.load_state_dict(model_state, strict=True, model_cfg=cfg.model)

In [18]:
checkpoint_folder = 'ckpts_v2'
checkpoint_fname = 'checkpoint62.pt'
save_dir = osp.join(checkpoint_folder, checkpoint_fname)
split = 'train'

checkpoint_path = Path(save_dir) 

In [36]:
import sys
from os import path
sys.path.append( path.dirname( path.dirname( path.abspath(os.curdir) ) ) )
sys.path.insert(1, '/home/jovyan/graphormer_v2/')

task = tasks.setup_task('graph_prediction')
model = task.build_model('graphormer')

AssertionError: Could not infer task type from graph_prediction. Available argparse tasks: dict_keys(['audio_pretraining', 'audio_finetuning', 'cross_lingual_lm', 'denoising', 'speech_to_text', 'text_to_speech', 'frm_text_to_speech', 'hubert_pretraining', 'language_modeling', 'legacy_masked_lm', 'masked_lm', 'multilingual_denoising', 'multilingual_language_modeling', 'multilingual_masked_lm', 'translation', 'multilingual_translation', 'online_backtranslation', 'semisupervised_translation', 'sentence_prediction', 'sentence_ranking', 'simul_speech_to_text', 'simul_text_to_text', 'speech_to_speech', 'translation_from_pretrained_bart', 'translation_from_pretrained_xlm', 'translation_lev', 'translation_multi_simple_epoch', 'dummy_lm', 'dummy_masked_lm', 'dummy_mt']). Available hydra tasks: dict_keys(['audio_pretraining', 'audio_finetuning', 'hubert_pretraining', 'language_modeling', 'masked_lm', 'multilingual_language_modeling', 'translation', 'sentence_prediction', 'simul_text_to_text', 't

In [37]:
data = preprocess_item(raw_data[0])
model_state = torch.load(checkpoint_path)["model"]
task = tasks.setup_task('graph_prediction')
model = task.build_model('graphormer')
model.load_state_dict(model_state, strict=True, model_cfg=cfg.model)
with torch.no_grad():
    model.eval()
    sample = utils.move_to_cuda(sample)
    y = model(**data)[:, 0, :].reshape(-1)
print(y)    

start find path dijkstra
dijkstra end with time 0.017045259475708008
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.04073143005371094


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [35]:
os.curdir

'.'

In [20]:
model_state = torch.load(checkpoint_path)["model"]

In [ ]:
model.load_state_dict(model_state, strict=True, model_cfg=cfg.model)

NameError: name 'model' is not defined

In [ ]:
# load dataset
split = args.split
task.load_dataset(split)
batch_iterator = task.get_batch_iterator(
    dataset=task.dataset(split),
    max_tokens=cfg.dataset.max_tokens_valid,
    max_sentences=cfg.dataset.batch_size_valid,
    max_positions=utils.resolve_max_positions(
        task.max_positions(),
        model.max_positions(),
    ),
    ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    required_batch_size_multiple=cfg.dataset.required_batch_size_multiple,
    seed=cfg.common.seed,
    num_workers=cfg.dataset.num_workers,
    epoch=0,
    data_buffer_size=cfg.dataset.data_buffer_size,
    disable_iterator_cache=False,
)
itr = batch_iterator.next_epoch_itr(
    shuffle=False, set_dataset_epoch=False
)
progress = progress_bar.progress_bar(
    itr,
    log_format=cfg.common.log_format,
    log_interval=cfg.common.log_interval,
    default_log_format=("tqdm" if not cfg.common.no_progress_bar else "simple")
)

# infer
y_pred = []
y_true = []
with torch.no_grad():
    model.eval()
    for i, sample in enumerate(progress):
        sample = utils.move_to_cuda(sample)
        y = model(**sample["net_input"])[:, 0, :].reshape(-1)
        y_pred.extend(y.detach().cpu())

In [3]:
data = list()
for i in range(0, 10):
    upd_data = preprocess_item(raw_data[i])
    upd_data.idx = i
    coll_data = collator_graphormer([upd_data])
    data.append(coll_data)


In [7]:
for key in data[0].keys():
    print(key, data[0][key].size())

idx torch.Size([1])
attn_bias torch.Size([1, 54, 54])
attn_edge_type torch.Size([1, 53, 53, 1])
spatial_pos torch.Size([1, 53, 53])
in_degree torch.Size([1, 53])
out_degree torch.Size([1, 53])
x torch.Size([1, 53, 13])
edge_input torch.Size([1, 53, 53, 1, 1])
y torch.Size([1])


In [8]:
data_raw = list()
for i in range(0, 10):
    upd_data = preprocess_item(raw_data[i])
    upd_data.idx = i
    data_raw.append(upd_data)


In [9]:
for key in data_raw[0].keys():
    print(key, data_raw[0][key].size())

TypeError: 'list' object is not callable

In [20]:
data_raw[0]

Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], idx=0, in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [17]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
name = 'collat'
torch.save(data_raw, osp.join(update_dir, f'{split}_{name}.pt'))

In [13]:
data[0]

In [18]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
name = 'raw'
torch.save(data, osp.join(update_dir, f'{split}_{name}.pt'))

In [19]:
data

[{'idx': tensor([0]),
  'attn_bias': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., -inf, -inf],
           [0., -inf, 0.,  ..., -inf, -inf, -inf],
           ...,
           [0., -inf, 0.,  ..., 0., -inf, -inf],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., -inf, 0.]]]),
  'attn_edge_type': tensor([[[[0],
            [0],
            [0],
            ...,
            [0],
            [0],
            [0]],
  
           [[0],
            [0],
            [0],
            ...,
            [0],
            [0],
            [0]],
  
           [[0],
            [0],
            [0],
            ...,
            [0],
            [0],
            [0]],
  
           ...,
  
           [[0],
            [3],
            [0],
            ...,
            [0],
            [0],
            [0]],
  
           [[0],
            [0],
            [0],
            ...,
            [0],
            [0],
            [3]],
  
           [[0],

In [28]:
raw_data[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [29]:
update_data = preprocess_item(raw_data[0])

In [30]:
update_data.idx = 1

In [44]:
update_data

Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], idx=1, in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [43]:
update_data['edge_input'][:, :, :1, :].size()

torch.Size([53, 53, 1, 1])

In [32]:
col_data = collator_graphormer([update_data])

In [33]:
for key in col_data.keys():
    print(key, col_data[key].size())

idx torch.Size([1])
attn_bias torch.Size([1, 54, 54])
attn_edge_type torch.Size([1, 53, 53, 1])
spatial_pos torch.Size([1, 53, 53])
in_degree torch.Size([1, 53])
out_degree torch.Size([1, 53])
x torch.Size([1, 53, 13])
edge_input torch.Size([1, 53, 53, 20, 1])
y torch.Size([1])


In [41]:
col_data['edge_input'][:, :, :1, :].size()

torch.Size([1, 53, 1, 20, 1])

In [ ]:
root = osp.join('dataset', 'omsk')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
split = 'train'
a = torch.load(update_dir + '/' + f'{split}.pt')